In [4]:
import io
import base64
import pandas as pd
import numpy as np
from helper_fcts import preprocess, get_poi_mas, rps
from keras import backend as K
from sklearn.preprocessing import scale, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVR, SVC
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.utils.vis_utils import model_to_dot
from xgboost import XGBClassifier, XGBRegressor
import matplotlib.pyplot as plt
from scipy.interpolate import spline
from IPython.display import SVG, HTML
from operator import itemgetter
from scipy.stats import poisson
from mpl_toolkits.mplot3d import Axes3D
from sklearn.externals import joblib

from main import modelling

In [89]:
df = pd.concat([modelling(pd.read_csv("data/D" + str(i) + ".csv"), 18) for i in range(8,19)])

C:\Users\Konny\Anaconda3\lib\site-packages\scipy\stats\_discrete_distns.py:493: RuntimeWarning: invalid value encountered in greater_equal
  return mu >= 0


### 4.2 Random Forest

In [32]:
columns_mit_fe  = ['FTR', 'season', 'H_avgGD', 'A_avgGD', 'H_avgG', 'A_avgG', 'H_avgG_c', 'A_avgG_c', 'H_avgST', 'A_avgST', 'H_avgST_c', 'A_avgST_c', 'H_GoalDiff_last', 'A_GoalDiff_last', 'H_xG_PoiMas', 'A_xG_PoiMas', 'H_Form_Tot4', 'A_Form_Tot4','H_Def_Rat', 'H_Off_Rat', 'A_Def_Rat', 'A_Off_Rat', "H_prob_odds", "D_prob_odds", "A_prob_odds"]
data2 = df[df["round"] > 10].reset_index(drop=True)
data = data2.loc[:,columns_mit_fe]
data.dropna(inplace=True)
X_train = data[data["season"]<18].iloc[:,2:]
X_test = data[data["season"]==18].iloc[:,2:]
y_train = data[data["season"]<18].loc[:,"FTR"]
y_test = data[data["season"]==18].loc[:,"FTR"]

X = data.iloc[:,2:]
y = data.loc[:,"FTR"]

In [33]:
clf = RandomForestClassifier(n_estimators=50,max_depth=4)

- training data: seasons 2008 - 2017
- test data: season 2018

In [34]:
clf.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=4, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

# ACHTUNG ÄNDERN!!!

In [35]:
joblib.dump(clf, 'rdf_clf_eng3.joblib')

['rdf_clf_eng3.joblib']

### 4.3 XGBoost

In [36]:
model = XGBClassifier(n_estimators=50, max_depth=4)
model.fit(X, y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=4, min_child_weight=1, missing=None, n_estimators=50,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

# ACHTUNG ÄNDERN!!!

In [37]:
joblib.dump(model, 'xgb_clf_eng3.joblib')

['xgb_clf_eng3.joblib']

### 4.4 Artificial Neural Network

In [90]:
columns_mit_fe  = ['Home', 'Draw', 'Away', 'season', 'H_avgGD', 'A_avgGD', 'H_avgG', 'A_avgG', 'H_avgG_c', 'A_avgG_c', 'H_avgST', 'A_avgST', 'H_avgST_c', 'A_avgST_c', 'H_GoalDiff_last', 'A_GoalDiff_last', 'H_xG_PoiMas', 'A_xG_PoiMas', 'H_Form_Tot4', 'A_Form_Tot4','H_Def_Rat', 'H_Off_Rat', 'A_Def_Rat', 'A_Off_Rat', "H_prob_odds", "D_prob_odds", "A_prob_odds"]

def get_season(df, season):
    X_train = df[df["season"] < season].iloc[:,4:]
    X_test = df[df["season"] == season].iloc[:,4:]
    y_train = df[df["season"] < season].iloc[:,:3]
    y_test = df[df["season"] == season].iloc[:,:3]
    return X_train, X_test, y_train, y_test

data2 = df[df["round"] > 10].reset_index(drop=True)
data = data2[columns_mit_fe]
data.dropna(inplace=True)

X = data.iloc[:,4:]
y = data.iloc[:,:3]


C:\Users\Konny\Anaconda3\lib\site-packages\ipykernel\__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


# ACHTUNG ÄNDERN!

In [91]:
scaler = StandardScaler()
scaler.fit(X)

#joblib.dump(scaler, 'scaler_eng3.joblib')

X = scaler.transform(X)

In [92]:
def loss_rps(y_true, y_pred):
    prob_h = y_pred[:, 0]
    prob_d = y_pred[:, 1]
    home = y_true[:, 0]
    draw = y_true[:, 1]

    step1 = prob_h - home
    step2 = prob_d - draw
    summe = step1 + step2
    return (step1 ** 2 + summe ** 2) / 2

In [93]:
K.clear_session()

model = Sequential()
model.add(Dense(128, kernel_initializer="he_normal", input_shape=(23,)))
model.add(Activation('tanh'))
model.add(Dropout(0.5))
model.add(Dense(64, kernel_initializer="he_normal"))
model.add(Activation('elu'))
model.add(Dropout(0.5))
model.add(Dense(32, kernel_initializer="he_normal"))
model.add(Activation('elu'))
model.add(Dropout(0.5))
model.add(Dense(32, kernel_initializer="he_normal"))
model.add(Activation('elu'))
model.add(Dropout(0.5))
model.add(Dense(3, kernel_initializer="he_normal"))
model.add(Activation('softmax'))

model.compile(loss=loss_rps, optimizer='adam', metrics=['accuracy'])

In [94]:
model.fit(X, y, epochs=100)

Epoch 1/100
2376/2376 [==============================] - 1s 226us/step - loss: 0.3129 - acc: 0.3834
Epoch 2/100
2376/2376 [==============================] - 0s 78us/step - loss: 0.2976 - acc: 0.3986
Epoch 3/100
2376/2376 [==============================] - 0s 77us/step - loss: 0.2813 - acc: 0.3990
Epoch 4/100
2376/2376 [==============================] - 0s 76us/step - loss: 0.2712 - acc: 0.4091
Epoch 5/100
2376/2376 [==============================] - 0s 68us/step - loss: 0.2693 - acc: 0.4171
Epoch 6/100
2376/2376 [==============================] - 0s 70us/step - loss: 0.2567 - acc: 0.4217
Epoch 7/100
2376/2376 [==============================] - 0s 75us/step - loss: 0.2487 - acc: 0.4213
Epoch 8/100
2376/2376 [==============================] - 0s 84us/step - loss: 0.2426 - acc: 0.4141
Epoch 9/100
2376/2376 [==============================] - 0s 92us/step - loss: 0.2326 - acc: 0.4238
Epoch 10/100
2376/2376 [==============================] - 0s 89us/step - loss: 0.2308 - acc: 0.4364
Epoch 11

Epoch 83/100
2376/2376 [==============================] - 0s 59us/step - loss: 0.2031 - acc: 0.5185
Epoch 84/100
2376/2376 [==============================] - 0s 58us/step - loss: 0.2002 - acc: 0.5223
Epoch 85/100
2376/2376 [==============================] - 0s 75us/step - loss: 0.2047 - acc: 0.5147
Epoch 86/100
2376/2376 [==============================] - 0s 73us/step - loss: 0.2014 - acc: 0.5118
Epoch 87/100
2376/2376 [==============================] - 0s 60us/step - loss: 0.2028 - acc: 0.5139
Epoch 88/100
2376/2376 [==============================] - 0s 65us/step - loss: 0.2024 - acc: 0.5168
Epoch 89/100
2376/2376 [==============================] - 0s 63us/step - loss: 0.2030 - acc: 0.5101
Epoch 90/100
2376/2376 [==============================] - 0s 63us/step - loss: 0.2007 - acc: 0.5248
Epoch 91/100
2376/2376 [==============================] - 0s 68us/step - loss: 0.2019 - acc: 0.5126
Epoch 92/100
2376/2376 [==============================] - 0s 63us/step - loss: 0.2026 - acc: 0.5152


# ACHTUNG ÄNDERN!

In [95]:
model.save('ann_clf_ger.h5')

## 6. Predicting Goals

#### XGB Regressor

In [44]:
columns_mit_fe  = ['FTHG', 'FTAG', 'season', 'H_avgGD', 'A_avgGD', 'H_avgG', 'A_avgG', 'H_avgG_c', 'A_avgG_c', 'H_avgST', 'A_avgST', 'H_avgST_c', 'A_avgST_c', 'H_GoalDiff_last', 'A_GoalDiff_last', 'H_xG_PoiMas', 'A_xG_PoiMas', 'H_Form_Tot4', 'A_Form_Tot4','H_Def_Rat', 'H_Off_Rat', 'A_Def_Rat', 'A_Off_Rat', "H_prob_odds", "D_prob_odds", "A_prob_odds"]
def get_season(df, season):
    X_train = df[df["season"] < season].iloc[:,2:]
    X_test = df[df["season"] == season].iloc[:,2:]
    y_train_hg = df[df["season"] < season].iloc[:,0]
    y_test_hg = df[df["season"] == season].iloc[:,0]
    y_train_ag = df[df["season"] < season].iloc[:,1]
    y_test_ag = df[df["season"] == season].iloc[:,1]
    return X_train, X_test, y_train_hg, y_test_hg, y_train_ag, y_test_ag

data2 = df[df["round"] > 10].reset_index(drop=True)
data = data2[columns_mit_fe]
data.dropna(inplace=True)

X = data.iloc[:,3:]
y_h = data.iloc[:,0]
y_a = data.iloc[:,1]

# X_train, X_test, y_train_hg, y_test_hg, y_train_ag, y_test_ag = get_season(data, 18)

# del X_train["season"]
# del X_test["season"]

C:\Users\Konny\Anaconda3\lib\site-packages\ipykernel\__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [45]:
#home
model_h = XGBRegressor(n_estimators=50, max_depth=4)
model_h.fit(X, y_h)
#xgb_home = model_h.predict(X_test)

#away
model_a = XGBRegressor(n_estimators=50, max_depth=4)
model_a.fit(X, y_a)
#xgb_away = model_a.predict(X_test)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=4, min_child_weight=1, missing=None, n_estimators=50,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

# ACHTUNG ÄNDERN!

In [46]:
joblib.dump(model_h, 'xgb_reg_eng3_h.joblib')
joblib.dump(model_a, 'xgb_reg_eng3_a.joblib')

['xgb_reg_eng3_a.joblib']

### SVR

In [47]:
#columns_mit_fe  = ['FTHG', 'FTAG', 'season', 'H_avgGD', 'A_avgGD', 'H_avgST', 'A_avgST', 'H_avgST_c', 'A_avgST_c', 'H_GoalDiff_last', 'A_GoalDiff_last', 'H_xG_PoiMas', 'A_xG_PoiMas', 'H_Form_Tot4', 'A_Form_Tot4', 'H_prob_odds', 'D_prob_odds', 'A_prob_odds']
columns_mit_fe  = ['FTHG', 'FTAG', 'season', 'H_avgGD', 'A_avgGD', 'H_avgG', 'A_avgG', 'H_avgG_c', 'A_avgG_c', 'H_avgST', 'A_avgST', 'H_avgST_c', 'A_avgST_c', 'H_GoalDiff_last', 'A_GoalDiff_last', 'H_xG_PoiMas', 'A_xG_PoiMas', 'H_Form_Tot4', 'A_Form_Tot4','H_Def_Rat', 'H_Off_Rat', 'A_Def_Rat', 'A_Off_Rat', "H_prob_odds", "D_prob_odds", "A_prob_odds"]

def get_season(df, season):
    X_train = df[df["season"] < season].iloc[:,2:]
    X_test = df[df["season"] == season].iloc[:,2:]
    y_train_hg = df[df["season"] < season].iloc[:,0]
    y_test_hg = df[df["season"] == season].iloc[:,0]
    y_train_ag = df[df["season"] < season].iloc[:,1]
    y_test_ag = df[df["season"] == season].iloc[:,1]
    return X_train, X_test, y_train_hg, y_test_hg, y_train_ag, y_test_ag

data2 = df[df["round"] > 10].reset_index(drop=True)
data = data2[columns_mit_fe]
data.dropna(inplace=True)

X_train, X_test, y_train_hg, y_test_hg, y_train_ag, y_test_ag = get_season(data, 18)

X = data.iloc[:,3:]
y_h = data.iloc[:,0]
y_a = data.iloc[:,1]

# del X_train["season"]
# del X_test["season"]

scaler = StandardScaler()
scaler.fit(X)

X = scaler.transform(X)
# X_test = scaler.transform(X_test)

C:\Users\Konny\Anaconda3\lib\site-packages\ipykernel\__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [48]:
#home
model_h = SVR()
model_h.fit(X, y_h)
#svr_home = model_h.predict(X_test)

#away
model_a = SVR()
model_a.fit(X, y_a)
#svr_away = model_a.predict(X_test)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

# ACHTUNG ÄNDERN!

In [49]:
joblib.dump(model_h, 'svr_reg_eng3_h.joblib')
joblib.dump(model_a, 'svr_reg_eng3_a.joblib')

['svr_reg_eng3_a.joblib']

### Artificial Neural Network

In [50]:
columns_mit_fe  = ['FTHG', 'FTAG', 'season', 'H_avgGD', 'A_avgGD', 'H_avgG', 'A_avgG', 'H_avgG_c', 'A_avgG_c', 'H_avgST', 'A_avgST', 'H_avgST_c', 'A_avgST_c', 'H_GoalDiff_last', 'A_GoalDiff_last', 'H_xG_PoiMas', 'A_xG_PoiMas', 'H_Form_Tot4', 'A_Form_Tot4','H_Def_Rat', 'H_Off_Rat', 'A_Def_Rat', 'A_Off_Rat', "H_prob_odds", "D_prob_odds", "A_prob_odds"]

def get_season(df, season):
    X_train = df[df["season"] < season].iloc[:,3:]
    X_test = df[df["season"] == season].iloc[:,3:]
    y_train = df[df["season"] < season].iloc[:,:2]
    y_test = df[df["season"] == season].iloc[:,:2]
    return X_train, X_test, y_train, y_test


data2 = df[df["round"] > 10].reset_index(drop=True)
data = data2[columns_mit_fe]
data.dropna(inplace=True)

#X_train, X_test, y_train, y_test = get_season(data, 18)

X = data.iloc[:,3:]
y = data.iloc[:,:2]

scaler = StandardScaler()
scaler.fit(X)

X = scaler.transform(X)
#X_test = scaler.transform(X_test)

C:\Users\Konny\Anaconda3\lib\site-packages\ipykernel\__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [51]:
K.clear_session()

model = Sequential()
model.add(Dense(128, kernel_initializer="he_normal", input_shape=(23,)))
model.add(Activation('tanh'))
model.add(Dropout(0.5))
model.add(Dense(64, kernel_initializer="he_normal"))
model.add(Activation('elu'))
model.add(Dropout(0.5))
model.add(Dense(32, kernel_initializer="he_normal"))
model.add(Activation('elu'))
model.add(Dropout(0.5))
model.add(Dense(2, kernel_initializer="he_normal"))
model.add(Activation('linear'))

model.compile(loss="logcosh", optimizer='adam', metrics=['accuracy'])

In [52]:
model.fit(X, y, epochs=80)

Epoch 1/80
4752/4752 [==============================] - 1s 110us/step - loss: 0.9148 - acc: 0.5305
Epoch 2/80
4752/4752 [==============================] - 0s 60us/step - loss: 0.6379 - acc: 0.5513
Epoch 3/80
4752/4752 [==============================] - 0s 64us/step - loss: 0.5560 - acc: 0.5852
Epoch 4/80
4752/4752 [==============================] - 0s 59us/step - loss: 0.5183 - acc: 0.6061
Epoch 5/80
4752/4752 [==============================] - 0s 63us/step - loss: 0.4897 - acc: 0.6349
Epoch 6/80
4752/4752 [==============================] - 0s 60us/step - loss: 0.4809 - acc: 0.6374
Epoch 7/80
4752/4752 [==============================] - 0s 60us/step - loss: 0.4728 - acc: 0.6524
Epoch 8/80
4752/4752 [==============================] - 0s 61us/step - loss: 0.4629 - acc: 0.6481
Epoch 9/80
4752/4752 [==============================] - 0s 61us/step - loss: 0.4588 - acc: 0.6503
Epoch 10/80
4752/4752 [==============================] - 0s 63us/step - loss: 0.4602 - acc: 0.6515
Epoch 11/80
4752/4

# ACHTUNG ÄNDERN!

In [53]:
model.save('ann_reg_eng3.h5')

In [1]:
from prediction import get_predictions

C:\Users\Konny\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
eng2 = get_predictions("eng2")

C:\Users\Konny\Anaconda3\lib\site-packages\scipy\stats\_discrete_distns.py:493: RuntimeWarning: invalid value encountered in greater_equal
  return mu >= 0


In [11]:
eng2[["HomeTeam", "AwayTeam", "H_prob_odds", "D_prob_odds", "A_prob_odds", "BbAHh", "Hc_avg", "Hc_Diff_avg"]]

,HomeTeam,AwayTeam,H_prob_odds,D_prob_odds,A_prob_odds,BbAHh,Hc_avg,Hc_Diff_avg
0,Birmingham,Reading,0.542915,0.263186,0.193898,-1.00,-0.937698,-0.062302
1,Middlesbrough,Rotherham,0.673659,0.213932,0.112409,-1.00,-1.413674,0.413674
2,Millwall,Wigan,0.437446,0.283361,0.279193,-0.25,-0.025852,-0.224148
3,Norwich,Aston Villa,0.420456,0.285355,0.294189,-0.25,-0.360187,0.110187
4,QPR,Sheffield Weds,0.467184,0.280587,0.252230,-0.25,-0.538725,0.288725
5,Sheffield United,Stoke,0.486190,0.275602,0.238209,-0.75,-0.611639,-0.138361
6,Swansea,Blackburn,0.423749,0.292061,0.284191,-0.25,-0.158933,-0.091067
7,Bolton,Nott'm Forest,0.274530,0.296905,0.428565,0.25,0.503948,-0.253948
8,Bristol City,Hull,0.512729,0.268711,0.218560,-0.75,-0.720944,-0.029056
9,Leeds,Ipswich,0.667106,0.210978,0.121916,-1.00,-1.486125,0.486125


In [7]:
eng3 = get_predictions("eng3")

C:\Users\Konny\Anaconda3\lib\site-packages\scipy\stats\_discrete_distns.py:493: RuntimeWarning: invalid value encountered in greater_equal
  return mu >= 0


In [10]:
eng3[["HomeTeam", "AwayTeam", "H_prob_odds", "D_prob_odds", "A_prob_odds", "BbAHh", "Hc_avg", "Hc_Diff_avg"]]

,HomeTeam,AwayTeam,H_prob_odds,D_prob_odds,A_prob_odds,BbAHh,Hc_avg,Hc_Diff_avg
0,Blackpool,Scunthorpe,0.500478,0.264297,0.235225,-0.75,-0.967130,0.217130
1,Bradford,Coventry,0.294305,0.287127,0.418568,0.25,-0.081580,0.331580
2,Bristol Rvs,AFC Wimbledon,0.389171,0.291577,0.319252,-0.25,-0.204802,-0.045198
3,Charlton,Oxford,0.492996,0.250431,0.256573,-0.50,-0.316180,-0.183820
4,Doncaster,Sunderland,0.382001,0.284764,0.333235,-0.25,0.043683,-0.293683
5,Luton,Accrington,0.575502,0.238339,0.186159,-1.00,-1.111672,0.111672
6,Peterboro,Fleetwood Town,0.450019,0.256978,0.293003,-0.25,-0.236156,-0.013844
7,Plymouth,Gillingham,0.387373,0.275238,0.337389,-0.25,-0.225371,-0.024629
8,Portsmouth,Burton,0.531713,0.268895,0.199392,-0.75,-0.871075,0.121075
9,Shrewsbury,Barnsley,0.277214,0.293674,0.429112,0.25,0.304211,-0.054211


In [9]:
eng3.columns

Index(['HomeTeam', 'AwayTeam', 'H_prob_odds', 'D_prob_odds', 'A_prob_odds',
       'BbAHh', 'H_pred_avg', 'D_pred_avg', 'A_pred_avg', 'Hc_avg',
       'Hc_Diff_avg', 'H_pred_Rdf', 'H_pred_Xgb', 'H_pred_Ann', 'D_pred_Rdf',
       'D_pred_Xgb', 'D_pred_Ann', 'A_pred_Rdf', 'A_pred_Xgb', 'A_pred_Ann',
       'XGB_hc', 'SVR_hc', 'ANN_hc'],
      dtype='object')